In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df_train = pd.read_csv("cleaned_training_data")
df_train.shape

(24016, 16)

In [3]:
df_train.head()

,location_x,location_y,power_of_shot,knockout_match,game_season,distance_of_shot,is_goal,area_of_shot,shot_basics,range_of_shot,date_of_game,home/away,shot_id_number,lat/lng,match_id,remaining_time
0,-157.0,0.0,1.0,0.0,2000-01,35.0,0.0,Left Side(L),Mid Range,8-16 ft.,31-10-2000,MANU @ POR,2.0,"45.539131, -122.651648",20000012,10.366667
1,-101.0,135.0,1.0,0.0,2000-01,36.0,1.0,Left Side Center(LC),Mid Range,16-24 ft.,31-10-2000,MANU @ POR,3.0,"45.539131, -122.651648",20000012,7.750000
2,138.0,175.0,1.0,0.0,2000-01,42.0,0.0,Right Side Center(RC),Mid Range,16-24 ft.,31-10-2000,MANU @ POR,4.0,"45.539131, -122.651648",20000012,6.866667
3,0.0,0.0,2.0,0.0,2000-01,20.0,1.0,Center(C),Goal Area,Less Than 8 ft.,31-10-2000,MANU @ POR,5.0,"45.539131, -122.651648",20000012,42.956667
4,-145.0,-11.0,3.0,0.0,2000-01,34.0,0.0,Left Side(L),Mid Range,8-16 ft.,31-10-2000,MANU @ POR,6.0,"45.539131, -122.651648",20000012,9.533333


In [4]:
one_hot = pd.get_dummies(df_train["area_of_shot"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("area_of_shot" , axis=1, inplace=True)

In [5]:
one_hot = pd.get_dummies(df_train["range_of_shot"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("range_of_shot" , axis=1, inplace=True)

In [6]:
one_hot = pd.get_dummies(df_train["shot_basics"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("shot_basics" , axis=1 , inplace=True)

In [7]:
gk = df_train.groupby(["home/away" , "lat/lng"]).count()
index1 = []
index2 = []
p = pd.DataFrame(gk.index, columns=["hah"] )
for i in range(len(p)):
    ho = p["hah"][i][0]
    la = p["hah"][i][1]
    index1.append(ho)
    index2.append(la)
ind1 = pd.DataFrame(index1, columns=["index1"] )
ind2 = pd.DataFrame(index2, columns=["index2"] )
p = p.join(ind1)
p = p.join(ind2)

p.drop("hah" , axis=1 ,inplace=True)

gg = df_train.groupby(["home/away" , "lat/lng" , "is_goal"])
score = {}
df_train["form"] = df_train["home/away"]
for i in range(len(p)):
    zero = len(gg.get_group((p["index1"][i] , p["index2"][i] , 0.0)))
    ones = len(gg.get_group((p["index1"][i] , p["index2"][i] , 1.0)))
    score[p["index1"][i]] = (ones/zero)

for i in tqdm(range(len(df_train))):
    df_train["form"][i] = score[df_train["home/away"][i]]

In [9]:
df_train.drop("lat/lng" , axis=1,inplace=True)
#df_train.drop("home/away" , axis=1,inplace=True)
df_train.drop("date_of_game" , axis=1 , inplace=True)
df_train.drop("shot_id_number" ,axis=1 ,inplace=True)
df_train.drop("match_id" , axis=1 ,inplace=True)
df_train.drop("game_season" ,axis=1, inplace=True)

In [10]:
one_hot = pd.get_dummies(df_train["home/away"] , drop_first=True)
df_train = df_train.join(one_hot)
df_train.drop("home/away" , axis=1 , inplace=True)

In [11]:
df_train.columns

Index(['location_x', 'location_y', 'power_of_shot', 'knockout_match',
       'distance_of_shot', 'is_goal', 'remaining_time', 'Left Side Center(LC)',
       'Left Side(L)', 'Mid Ground(MG)', 'Right Side Center(RC)',
       'Right Side(R)', '24+ ft.', '8-16 ft.', 'Back Court Shot',
       'Less Than 8 ft.', 'Goal Line', 'Left Corner', 'Mid Ground Line',
       'Mid Range', 'Penalty Spot', 'Right Corner', 'form', 'MANU @ BKN',
       'MANU @ BOS', 'MANU @ CHA', 'MANU @ CHH', 'MANU @ CHI', 'MANU @ CLE',
       'MANU @ DAL', 'MANU @ DEN', 'MANU @ DET', 'MANU @ GSW', 'MANU @ HOU',
       'MANU @ IND', 'MANU @ LAC', 'MANU @ MEM', 'MANU @ MIA', 'MANU @ MIL',
       'MANU @ MIN', 'MANU @ NJN', 'MANU @ NOH', 'MANU @ NOK', 'MANU @ NOP',
       'MANU @ NYK', 'MANU @ OKC', 'MANU @ ORL', 'MANU @ PHI', 'MANU @ PHO',
       'MANU @ PHX', 'MANU @ POR', 'MANU @ SAC', 'MANU @ SAS', 'MANU @ SEA',
       'MANU @ TOR', 'MANU @ UTA', 'MANU @ UTH', 'MANU @ VAN', 'MANU @ WAS',
       'MANU vs. ATL', 'MANU vs.

In [12]:
df_train

,location_x,location_y,power_of_shot,knockout_match,distance_of_shot,is_goal,remaining_time,Left Side Center(LC),Left Side(L),Mid Ground(MG),...,MANU vs. PHX,MANU vs. POR,MANU vs. SAC,MANU vs. SAN,MANU vs. SAS,MANU vs. SEA,MANU vs. TOR,MANU vs. UTA,MANU vs. VAN,MANU vs. WAS
0,-157.0,0.0,1.0,0.0,35.0,0.0,10.366667,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-101.0,135.0,1.0,0.0,36.0,1.0,7.750000,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,138.0,175.0,1.0,0.0,42.0,0.0,6.866667,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,2.0,0.0,20.0,1.0,42.956667,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-145.0,-11.0,3.0,0.0,34.0,0.0,9.533333,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0.0,0.0,3.0,0.0,20.0,1.0,8.866667,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,-65.0,0.0,3.0,0.0,32.0,1.0,6.200000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,-33.0,0.0,3.0,0.0,32.0,0.0,3.600000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,-94.0,238.0,3.0,0.0,45.0,0.0,1.933333,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,121.0,127.0,1.0,0.0,37.0,1.0,11.000000,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [13]:
x = df_train.iloc[: , df_train.columns != "is_goal"].values
y = df_train.iloc[: , df_train.columns == "is_goal"].values

In [14]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=seed)

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
model = XGBClassifier(max_depth=2 ,learning_rate=0.05)
model.fit(X_train , y_train)

y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test , y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [20]:
model.feature_importances_

array([0.03483482, 0.03258698, 0.02894035, 0.        , 0.3326154 ,
       0.06832937, 0.        , 0.01739147, 0.        , 0.        ,
       0.02799268, 0.08900113, 0.00813899, 0.        , 0.21262981,
       0.        , 0.        , 0.        , 0.02115399, 0.        ,
       0.        , 0.03354813, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.0358599 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.02324858, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [21]:
temp = df_train.iloc[: , df_train.columns != "is_goal"]

In [23]:
importances = pd.DataFrame({"columns":temp.columns , "importances":model.feature_importances_})

importances.sort_values("importances" ,ascending=False)

,columns,importances
4,distance_of_shot,0.332615
14,Less Than 8 ft.,0.212630
11,24+ ft.,0.089001
5,remaining_time,0.068329
31,MANU @ GSW,0.035860
0,location_x,0.034835
75,MANU vs. MIN,0.033728
21,form,0.033548
1,location_y,0.032587
2,power_of_shot,0.028940


In [ ]:
temp = pd.read_csv("cleaned_test_data")
temp["shot_id_number"] = temp["shot_id_number"].astype('int64')
result = pd.DataFrame(data={"shot_id_number" : temp["shot_id_number"], "is_goal" : y_pred[:, 1]})
result.to_csv("1stAttempt.csv", index=False)